<a href="https://colab.research.google.com/github/anujsahani01/English-Marathi-Translation/blob/main/FineTuned_AI4Bharat(EN_MR).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!pip install datasets
!pip install --upgrade accelerate
!pip install wandb
!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install evaluate
!pip install rouge_score
! pip install sacremoses
! pip install sacrebleu

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.30.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? Y
  Successfully uninstalled transformers-4.30.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 108.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d mathurinache/samanantar

samanantar.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

with zipfile.ZipFile('/content/samanantar.zip') as zf:
    zf.extract('final_data/en-mr/train.mr', 'data')

with zipfile.ZipFile('/content/samanantar.zip') as zf:
    zf.extract('final_data/en-mr/train.en', 'data')

In [ ]:
import pandas as pd

In [ ]:
english_text = ''
marathi_text = ''

with open('/content/data/final_data/en-mr/train.en' , 'r', encoding = 'utf-8') as f:
  english_text = f.read()

with open('/content/data/final_data/en-mr/train.mr' , 'r', encoding = 'utf-8') as f:
  marathi_text = f.read()

In [ ]:
english = []
marathi = []
english = english_text.split('\n')
marathi = marathi_text.split('\n')

In [ ]:
df1 = pd.DataFrame()
df1.column = ['inputs', 'targets']

<ipython-input-11-e9738c3977d1>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df1.column = ['inputs', 'targets']


In [ ]:

print(len(english), len(marathi))

3322057 3322057


In [ ]:
df1['inputs'] = english
df1['targets'] = marathi

In [ ]:
import pandas as pd
df2 = pd.read_csv('/content/data.csv')

In [ ]:
df3 = pd.read_csv('/content/data1.csv')
df4 = pd.read_csv('/content/data2.csv')
# df5 = pd.read_csv('/content/data3.csv')

In [ ]:
frames = [df1, df2, df3, df4]
df_data = pd.concat(frames)

In [ ]:
df_data.columns = ['english', 'marathi']

In [ ]:
df_data.dropna(inplace= True)

In [ ]:
df_data.head(10)

,english,marathi
0,Next few months are really crucial for us.,पुढील तीन महिने आमच्यासाठी खूप महत्त्वपूर्ण आहेत.
1,"Sharad Pawar, Supriya Sule, Ajit Pawar, Praful...","या बैठकील पक्षाध्यक्ष शरद पवार, केंद्रीय मंत्र..."
2,"""Kanwal Singh Chauhan, the president of Sonepa...","वृत्तसंस्थेने दिलेल्या माहितीनुसार, विरोध करण्..."
3,His demise is a huge loss for India.,त्यांच्या मृत्यूमुळे भारताचे खूप मोठे नुकसान झ...
4,Photographer and raconteur.,छायाचित्रकार आणि छायाचित्रकार.
5,He has won several competitions at the state a...,राज्य व राष्ट्रीय पातळीवरील अनेक मल्ल त्यांनी ...
6,This is a worrying increase.,ही वाढ चिंताजनक तर आहेच.
7,Most of the victims were Dalits.,त्यातील बहुतांशी घटनांत पीडित व्यक्ती या दलित ...
8,This has shocked the Congress.,त्यामुळे काँग्रेसला मोठा धक्का बसला.
9,This was not the idea.,हा कल्पनाविलास नव्हता.


In [ ]:
len(df_data)

3400084

In [ ]:
l = len(df_data)
split = int(0.75*l)
df_train = df_data[:split]
# shuffle the DataFrame rows
df_train = df_train.sample(frac = 1)
df_train.reset_index(inplace = True)
df_train.drop('index', axis = 1, inplace = True)

df_test = df_data[split:]
df_test = df_test.sample(frac = 1)
df_test.reset_index(inplace = True)
df_test.drop('index', axis = 1, inplace = True);

In [ ]:
df_train.head(10)

,english,marathi
0,Director Karan Johar and producer Boney Kapoor...,निर्माता आणि दिग्दर्शक करण जोहर यानेही बाबा बन...
1,"After hearing the arguments of both sides, the...",दोन्ही पक्षांची बाजू ऐकल्यानंतर सर्वोच्च न्याय...
2,The traffic was restored by the evening.,रात्री ही वाहतूक पूर्ववत करण्यात आली.
3,Material benefits,साहित्य फायदे
4,Police have registered a case on the accident.,अपघाताची कवठेमहांकाळ पोलिसांत नोंद झाली आहे.
5,This was informed by Finance Minister Nirmala ...,अर्थमंत्री निर्मला सितारमण यांनी संसदेतील भाषण...
6,Additional responsibilities in job too might b...,नोकरीमध्ये अतिरिक्त जबाबदारी मिळू शकते.
7,The Congress won only one seat and the NCP four.,राष्ट्रवादीने चार जागा जिंकल्या तर काँग्रेसला ...
8,The media cannot be muzzled.,माध्यमांचे काही होऊ शकत नाही.
9,Forest officials and police personnel reached ...,वनविभागाचे कर्मचारी आणि पालिका अधिकारी घटनास्थ...


In [ ]:
print(len(df_train), len(df_test))

2550063 850021


In [ ]:
df_train['length'] = df_train['english'].apply(
    lambda row: min(len(row.split(" ")), len(row)) if isinstance(row, str) else None
)

In [ ]:
print(df_train['length'].min())
print(df_train['length'].max())
print(df_train['length'].mean())

1
10263
10.831284560420665


In [ ]:
df_train.drop(['length'], axis = 1, inplace = True);

In [ ]:
df_train = df_train.loc[(df_train['english'].str.len() <= 80) & (df_train['english'].str.len() >= 40)]

In [ ]:
df_test = df_test.loc[(df_test['english'].str.len() <= 80) & (df_test['english'].str.len() >= 40)]

In [ ]:
print(len(df_train), len(df_test))

882869 281254


In [ ]:
df_train.reset_index(inplace = True)
df_test.reset_index(inplace = True)

In [ ]:
df_train.drop(['index'], axis = 1, inplace = True)
df_test.drop(['index'], axis = 1, inplace= True)

In [ ]:
df_train.to_csv('train.csv', encoding = 'utf-8', index= False)

In [ ]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict({
    "train" : Dataset.from_pandas(df_train[:int(len(df_train)/2)]),
    "test" : Dataset.from_pandas(df_test[:int(len(df_test)/2)])
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 441434
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 140627
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!huggingface-cli login

In [ ]:
# from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from transformers import FSMTForConditionalGeneration, FSMTTokenizer, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', keep_accents=True, eos_token = '[CLS]', model_max_length = 512)
tokenizer.bos_token = tokenizer.eos_token

In [ ]:
out = tokenizer('She is not at home. She is in the garden.In the garden she plays with her friends.', text_target = 'ती घरी नाही. ती बागेत आहे.बागेत ती तिच्या मैत्रिणींसोबत खेळते.',  max_length= 50, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')
out1 = tokenizer(dataset['train']['english'][0], text_target =  dataset['train']['marathi'][0],  max_length= 20, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')

In [ ]:
out1

{'input_ids': tensor([[    2,  4548,     8, 82828, 55011, 10322,    24, 20650, 59575,   335,
         15988, 75454,   297,   271,  3368,     9,     3,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]), 'length': tensor([20]), 'labels': tensor([[     2,  13547,     86,  26882,  16896,    219,   7189,     75,  54838,
           4798,   1311,   5891, 158254, 121957,  13028,   3608,   4134,    343,
              9,      3]])}

In [ ]:
print(out['input_ids'])
print(out['attention_mask'])

tensor([[    2,   839,    54,   179,   117,  2279,     9,   839,    54,    26,
            11, 69284,     9,  1197,    11, 69284,   839, 21581,    90,   481,
         10574,     9,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])


In [ ]:
decoded_preds = tokenizer.batch_decode(out['labels'])
print(decoded_preds)

['[CLS] ती घरी नाही. ती बागेत आहे.बागेत ती तिच्या मैत्रिणींसोबत खेळते.[SEP]<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>']


In [ ]:
tokenizer

AlbertTokenizerFast(name_or_path='ai4bharat/indic-bert', vocab_size=200000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[CLS]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [ ]:
import numpy as np

In [ ]:
max_length = 70
context_length = 70

def tokenize(data):
  data = tokenizer(data['english'], text_target =  data['marathi'],  max_length= max_length, padding = 'max_length', truncation=True, return_length = True, return_tensors = 'pt')
  inp_ids = []
  if len(data['input_ids']) != 0:
    for length, ids in zip(data["length"], data["input_ids"]):
      inp_ids.append(ids)
    label = []
    for length, lb in zip(data["length"], data["labels"]):
      label.append(lb)
  return {"input_ids": inp_ids,
          'labels' : label}

In [ ]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns = dataset['train'].column_names,
)

Map:   0%|          | 0/441434 [00:00<?, ? examples/s]

Map:   0%|          | 0/140627 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 441434
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 140627
    })
})

In [ ]:
inp = tokenized_dataset['train']['input_ids']

In [ ]:
arr = []
df_final = pd.DataFrame()
df_final['tokenized_eng'] = inp

In [ ]:
df_final['length'] = df_final['tokenized_eng'].apply(
    lambda row: len(row)
)

In [ ]:
print(df_final['length'].min())
print(df_final['length'].max())
print(df_final['length'].min())

70
70
70


In [ ]:
out = tokenized_dataset['train'][0]

In [ ]:
print(out['labels'])
print(out['input_ids'])

[2, 13547, 86, 26882, 16896, 219, 7189, 75, 54838, 4798, 1311, 5891, 158254, 121957, 13028, 3608, 4134, 343, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 4548, 8, 82828, 55011, 10322, 24, 20650, 59575, 335, 15988, 75454, 297, 271, 3368, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
from transformers import AutoModel

In [ ]:
! pip install -U neptune transformers

In [ ]:
from getpass import getpass
from transformers.integrations import NeptuneCallback
import neptune
token=my_api_token = getpass("Enter your Neptune API token:")

Enter your Neptune API token:··········


In [ ]:
my_project = "anujsahani01/ITStudio"
run = neptune.init_run(
    project="anujsahani01/ITStudio",
    api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMzNmZWY5My0yOGY1LTRiZDItODA1MS05ZTM3MTc0MTI2NTYifQ==',
    source_files= ["/content/drive/MyDrive/AI4Bharat_optimized(FINALL).ipynb"]
)

<ipython-input-57-d9acc3b93ba8>:2: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  run = neptune.init_run(


https://app.neptune.ai/anujsahani01/ITStudio/e/MODEL-57


In [ ]:
run["train_dataset"].track_files("./train.csv")

In [ ]:
from transformers import AutoConfig, AlbertConfig, MBartConfig
model_checkpoint  = 'ai4bharat/indic-bert'
config = MBartConfig.from_pretrained(model_checkpoint, vocab_size=len(tokenizer), use_auth_token=True)
config.vocab_size = tokenizer.vocab_size

You are using a model of type albert to instantiate a model of type mbart. This is not supported for all configurations of models and can yield errors.


In [ ]:
from transformers.models.auto.modeling_auto import AutoModelForCausalLM, AutoModel, AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, config = config).to('cuda')

model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing MBartForConditionalGeneration: ['albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight', 'predictions.decoder.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight', 'albert.embeddings.word_embeddings.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.LayerNorm.bias', 'albert.pooler.bias', 'albert.embeddings.LayerNorm.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn_output.weight', 'sop_classifier.classifier.weight', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'predictions.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'albert.encoder.embedding_hidden_mapping_in.bias', 'predictions.LayerNorm.bias', 'albert.encoder.albert_layer_groups.0.albert_layers.0.ffn.weight

Embedding(200000, 768, padding_idx=1)

In [ ]:
model.config

MBartConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "down_scale_factor": 1,
  "dropout": 0.1,
  "embedding_size": 128,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "forced_eos_token_id": 2,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "init_std": 0.02,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "is_encoder_decoder": true,
  "max_position_embeddings": 512,
  "model_type": "mbart",
  "net_structure_type": 0,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 1,
  "scale_embedding"

In [ ]:
print(model)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(200000, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(200000, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(514, 768)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((76

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import evaluate
metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
  preds, labels = eval_preds
  # In case the model returns more than the prediction logits
  if isinstance(preds, tuple):
      preds = preds[0]

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

  # Replace -100s in the labels as we can't decode them
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # Some simple post-processing
  decoded_preds = [pred.strip() for pred in decoded_preds]
  decoded_labels = [[label.strip()] for label in decoded_labels]

  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  return {"bleu": result["score"]}

In [ ]:
params = {'lr':0.0005, 'max_steps': 8000, 'warmup_steps': 50, 'weight_decay': 0.01, 'per_device_train_batch_size': 12, 'per_device_eval_batch_size': 12, 'evaluation_strategy' :"no", 'save_total_limit' :1, 'num_train_epochs': 1, 'predict_with_generate' : True, 'fp16' : False, 'push_to_hub' : True, 'remove_unused_columns' :False, 'dropout': 0.5}

In [ ]:
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer

args =  Seq2SeqTrainingArguments(
    f"./finetuned_AI4Bharat_en_mr",
    evaluation_strategy= params['evaluation_strategy'],
    save_strategy="epoch",
    learning_rate= params['lr'] ,
    per_device_train_batch_size=params['per_device_train_batch_size'],
    per_device_eval_batch_size= params['per_device_eval_batch_size'],
    warmup_steps = params['warmup_steps'],
    weight_decay= params['weight_decay'],
    save_total_limit= params['save_total_limit'],
    num_train_epochs= params['num_train_epochs'],
    predict_with_generate= params['predict_with_generate'],
    fp16= params['fp16'],
    push_to_hub= params['push_to_hubacc'],
    max_steps = params['max_steps'],
    remove_unused_columns= params['remove_unused_columns'],
    report_to="neptune",
    # predict_with_generate = True
)

In [ ]:
trainer = Seq2SeqTrainer(
    model.to('cuda'),
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,)

/content/./finetuned_AI4Bharat_en_mr is already a clone of https://huggingface.co/anujsahani01/finetuned_AI4Bharat_en_mr. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
for cb in trainer.callback_handler.callbacks:
  if isinstance(cb, NeptuneCallback):
    trainer.callback_handler.remove_callback(cb)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.507700
1000,1.297200
1500,1.240900
2000,1.203000
2500,1.168900
3000,1.133100
3500,1.107700
4000,1.084100
4500,1.071700
5000,1.049000


Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


TrainOutput(global_step=8000, training_loss=1.115094051361084, metrics={'train_runtime': 6440.7353, 'train_samples_per_second': 14.905, 'train_steps_per_second': 1.242, 'total_flos': 9525810954240000.0, 'train_loss': 1.115094051361084, 'epoch': 0.22})

In [ ]:
metrics = trainer.state.log_history

In [ ]:
for key, value in params.items():
  run[key] = value

In [ ]:
lr = []
train_loss = []
steps =  []
epoch = []

In [ ]:
for i in range(len(metrics)-1):
  train_loss.append(metrics[i]['loss'])
  steps.append(metrics[i]['step'])
  lr.append(metrics[i]['learning_rate'])
  epoch.append(metrics[i]['epoch'])

In [ ]:
for i in range(len(train_loss)):
  run['train/loss'].append(train_loss[i])

In [ ]:
for i in range(len(epoch)):
  run['epoch'].append(epoch[i])

In [ ]:
for i in range(len(lr)):
  run["learning_rate"].append(lr[i])

In [ ]:
for i in range(len(steps)):
  run["steps"].append(steps[i])

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/1.46G [00:00<?, ?B/s]

To https://huggingface.co/anujsahani01/finetuned_AI4Bharat_en_mr
   8d6deef..954a26c  main -> main

   8d6deef..954a26c  main -> main

To https://huggingface.co/anujsahani01/finetuned_AI4Bharat_en_mr
   954a26c..a8420e2  main -> main

   954a26c..a8420e2  main -> main



'https://huggingface.co/anujsahani01/finetuned_AI4Bharat_en_mr/commit/954a26c161ddcaf9a0b4bbc93b0280bcc1436bf0'

In [ ]:
!  pip install sacremoses

In [ ]:
from transformers import AutoModelForCausalLM, TranslationPipeline, Text2TextGenerationPipeline, pipeline

model_ckpt = "anujsahani01/finetuned_AI4Bharat_en_mr"
model = AutoModelForSeq2SeqLM.from_pretrained("anujsahani01/finetuned_AI4Bharat_en_mr")
pipe = pipeline('text2text-generation', "anujsahani01/finetuned_AI4Bharat_en_mr")

In [ ]:
src_text = 'he is playing outside'
pred = pipe(src_text, num_return_sequences=1)
pred

[{'generated_text': ''}]

In [ ]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/anujsahani01/ITStudio/e/MODEL-57/metadata
